<a href="https://colab.research.google.com/github/baemsu/faq_code/blob/main/%5BFAQ%5D_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%EB%8D%94%EB%AF%B8%EB%B3%80%EC%88%98%EB%AA%A8%EB%8D%B8%EC%A0%80%EC%9E%A5%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**(Q) x에 대해 전처리에서 범주형데이터를 get_dummies를 통해 더미변수화 한후 이를 어떻게 저장하는지요?**

##[방법]
* 인공지능 모델을 API화 한후, 새로운 X(X가 범주형 데이터) 값을 가지고 예측할 때임
* X가 범주형 데이터 일 경우, 학습 모델은 더미변수화를 통해 학습이 이루어짐
* 이 학습모델를 로딩한여 신규 X를 넣을 경우, 더미변수화되지 않은 상태이므로, 예측이 불가함.
* 그리고 별도로 신규 X만으로 더미변수화 할경우, 모든 범주가 데이터셋에 있지 않기 때문에 역시 더미변수 Shape가 다름 (예: X가 색깔로 빨강, 파랑, 노랑이 있다고 가정했을때, 학습 데이터 셋으로 더미변수화 하면 color_red, color_blue, color_yellow가 생기지만, 신규 X로 더미변수화하면 하나의 변수만 생김)
* 기존의 get_dummies의 경우에는 학습데이터셋의 더미변수 셋들을 저장하지 못함. 
* 이를 해결할수 있는 방법이 OneHoeEncoder를 사용하면 해결됨


## Notation

```

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False)
data_cat = ohe.fit_transform(data[['범주형데이터']])
data_cat

#카테고리 확인 (범주형데이터의 카테고리들이 저장되어 있음
print(ohe.categories_[0])

#그리고나서 원본데이터에 부착함 (원래의 X를 없애고, 원핫인코딩한 데이터에서 컬럼명을 바꾼 내용으로 추가하여 부착함 

pd.concat([data.drop(columns=['범주형데이터']),pd.DataFrame(data_cat, columns=['범주형데이터명_' + str(col) for col in ohe.categories_[0]])], axis=1)
```





# **(예시) 단순회귀모형을 만들고 스케일러와 모델을 함께 저장한 후, 스케일러와 모델를 호출하여 신규 값을 예측.**

## (1) 일단 단순 회귀 모델을 만들어 봅니다.  스케일러 적용

In [1]:
#모델 생성
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd

data = {'x':['boy','boy','boy','girl','girl','girl','girl','adult','adult','adult'],'y':[110,214,313,456,556,687,789,820,950,1023]}

df = pd.DataFrame(data)



In [2]:
############# 더미변수화  ############################

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False)

# fit_transform은 train에만 사용하고 test에는 학습된 인코더에 fit만 해야한다

data_cat = ohe.fit_transform(df[['x']])
print(data_cat)

#카테고리 확인
print(ohe.categories_[0])

#더미 변수 처럼 컬럼을 생성함 (카테고리명으로)
#pd.DataFrame(data_cat, columns=['cat1_' + col for col in ohe.categories_[0]])

#그리고나서 원본데이터에 부착함
df_onehotcode = pd.concat([df.drop(columns=['x']),pd.DataFrame(data_cat, columns=['x_' + str(col) for col in ohe.categories_[0]])], axis=1)

###################################################


[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
['adult' 'boy' 'girl']


In [3]:
df_onehotcode

,y,x_adult,x_boy,x_girl
0,110,0.0,1.0,0.0
1,214,0.0,1.0,0.0
2,313,0.0,1.0,0.0
3,456,0.0,0.0,1.0
4,556,0.0,0.0,1.0
5,687,0.0,0.0,1.0
6,789,0.0,0.0,1.0
7,820,1.0,0.0,0.0
8,950,1.0,0.0,0.0
9,1023,1.0,0.0,0.0


In [4]:
x = df_onehotcode.drop(columns = ["y"], axis = 1)
y = df_onehotcode["y"]

In [5]:
x

,x_adult,x_boy,x_girl
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,0.0,1.0,0.0
3,0.0,0.0,1.0
4,0.0,0.0,1.0
5,0.0,0.0,1.0
6,0.0,0.0,1.0
7,1.0,0.0,0.0
8,1.0,0.0,0.0
9,1.0,0.0,0.0


In [6]:
y

0     110
1     214
2     313
3     456
4     556
5     687
6     789
7     820
8     950
9    1023
Name: y, dtype: int64

In [7]:

model = LinearRegression()
model.fit(x,y)

print("기울기: ", model.coef_[0])
print("절편: ", model.intercept_)


기울기:  -7.83324210282462e+16
절편:  7.833242102824712e+16


## (2) 원핫인코드모델과 학습모델을 모두 저장합니다.

In [8]:
import joblib
joblib.dump(model, './regression_model.pkl')
joblib.dump(ohe, "./ohe_x.pkl")

['./ohe_x.pkl']

## (3) model과 scaler를 호출하여 신규 x가 "boy" 일때 y값을 예측해봅니다.

In [9]:
# 저장된 스케일러와 모델을 호출합니다.
import joblib
loaded_model = joblib.load("./regression_model.pkl")
loaded_ohe = joblib.load("./ohe_x.pkl")

In [10]:
# 신규 x를 데이터프레임형태로 만듦니다.

new_x = {}
new_x= {'x':["boy"]}
new_x_df = pd.DataFrame(new_x)
print(new_x_df)


     x
0  boy


In [11]:
new_x_df["x"]

0    boy
Name: x, dtype: object

In [14]:
#새로운데이터의 범주형데이터를 변환하기
data_cat2 = loaded_ohe.transform(new_x_df[['x']])
data_concat = pd.concat([new_x_df.drop(columns=['x']),pd.DataFrame(data_cat2, columns=['x_' + str(col) for col in loaded_ohe.categories_[0]])], axis=1)

In [15]:
data_concat

,x_adult,x_boy,x_girl
0,0.0,1.0,0.0


In [16]:
new_y = loaded_model.predict(data_concat)
new_y

array([192.])